In [ ]:
#r "nuget: CsvHelper, 33.0.1" 
#r "nuget: Microsoft.Extensions.AI.Ollama, 9.3.0-preview.1.25114.11" 
#r "nuget: Microsoft.Extensions.VectorData.Abstractions, 9.0.0-preview.1.25078.1" 
#r "nuget: Microsoft.SemanticKernel.Connectors.InMemory, 1.40.1-preview" 

Installed Packages CsvHelper, 33.0.1 Microsoft.Extensions.AI.Ollama, 9.3.0-preview.1.25114.11 Microsoft.Extensions.VectorData.Abstractions, 9.0.0-preview.1.25078.1 Microsoft.SemanticKernel.Connectors.InMemory, 1.40.1-preview

In [ ]:
using System.Globalization;
using System.Text.RegularExpressions;
using CsvHelper;
using CsvHelper.Configuration;
using Microsoft.Extensions.AI;
using Microsoft.Extensions.VectorData;
using Microsoft.SemanticKernel.Connectors.InMemory;
using System.IO;

In [ ]:
IEmbeddingGenerator<string, Embedding<float>> generator =
    new OllamaEmbeddingGenerator(new Uri("http://localhost:11434/"), "nomic-embed-text");

In [ ]:
var king = await generator.GenerateEmbeddingVectorAsync("king");
var man = await generator.GenerateEmbeddingVectorAsync("man");
var woman = await generator.GenerateEmbeddingVectorAsync("woman");
var queen = await generator.GenerateEmbeddingVectorAsync("queen");

Console.WriteLine(king);

for (int i = 0; i < king.Length; i++)
{
    Console.WriteLine(king.Span[i]);
}

System.ReadOnlyMemory<Single>[384]
-0.059614934
0.050269842
-0.069633365
0.07963753
-0.046778742
0.00083347573
0.07913668
-0.01272995
0.05834293
-0.031435847
-0.07119536
-0.08490141
0.04229277
-0.03418061
-0.021509102
0.010418978
0.00021182121
0.023824686
-0.09131088
-0.040176284
-0.03444211
0.0097214
-0.046671037
0.061408512
-0.08005588
-0.010228573
-0.020874945
0.024942959
-0.010926009
-0.14766893
0.01143833
-0.09492554
0.034380767
0.032124028
-0.00274255
0.040804904
-0.035935752
-0.009372843
0.009514046
-0.0051336717
0.020491792
-0.026703183
-0.0049422015
-0.023381187
0.047574807
0.0885302
-0.00945836
-0.0068451543
0.08820478
0.025187299
-0.008862212
-0.010656514
0.018197732
-0.0004600204
0.029848548
-0.017845955
-0.04578923
-0.030988151
0.106108755
-0.0032199947
0.050381586
-0.0018183293
-0.019594017
0.035144463
0.0023181473
-0.05454632
0.045261823
0.0353719
-0.014533313
-0.031904742
-0.071364924
0.0072503677
0.07400964
0.024219628
-0.027050262
-0.04366743
-0.025872637
-0.02336381


In [ ]:
// Define some helper functions

static ReadOnlyMemory<float> SubtractVectors(ReadOnlyMemory<float> vector1, ReadOnlyMemory<float> vector2)
{
    var result = new float[vector1.Length];
    for (var i = 0; i < vector1.Length; i++)
    {
        result[i] = vector1.Span[i] - vector2.Span[i];
    }
    return new ReadOnlyMemory<float>(result);
}

static ReadOnlyMemory<float> AddVectors(ReadOnlyMemory<float> vector1, ReadOnlyMemory<float> vector2)
{
    var result = new float[vector1.Length];
    for (var i = 0; i < vector1.Length; i++)
    {
        result[i] = vector1.Span[i] + vector2.Span[i];
    }
    return new ReadOnlyMemory<float>(result);
}


decimal CosineSimilairty(ReadOnlyMemory<float> vector1, ReadOnlyMemory<float> vector2)
{
    if (vector1.Length != vector2.Length)
        throw new ArgumentException("Vectors must be of the same length");

    float dotProduct = 0;
    float magnitude1 = 0;
    float magnitude2 = 0;

    for (int i = 0; i < vector1.Length; i++)
    {
        dotProduct += vector1.Span[i] * vector2.Span[i];
        magnitude1 += vector1.Span[i] * vector1.Span[i];
        magnitude2 += vector2.Span[i] * vector2.Span[i];
    }

    magnitude1 = (float)Math.Sqrt(magnitude1);
    magnitude2 = (float)Math.Sqrt(magnitude2);

    if (magnitude1 == 0 || magnitude2 == 0)
        return 0;

    return (decimal)(dotProduct / (magnitude1 * magnitude2));
}

static decimal SimilarityInPercent(decimal cosineSimilairty)
{
    return (decimal)(cosineSimilairty + 1) / 2 * 100;
}

In [ ]:

var kingMinusManPlusWoman = AddVectors(SubtractVectors(king, man), woman);

Console.WriteLine($"Similarity between king - man + woman and queen: {CosineSimilairty(kingMinusManPlusWoman, queen)}, {SimilarityInPercent(CosineSimilairty(kingMinusManPlusWoman, queen))}%");
Console.WriteLine($"Similarity between king and king: {CosineSimilairty(king, king)}, {SimilarityInPercent(CosineSimilairty(king, king))}%");
Console.WriteLine($"Similarity between King and Man: {CosineSimilairty(king, man)}, {SimilarityInPercent(CosineSimilairty(king, man))}%");
Console.WriteLine($"Similarity between King and Woman: {CosineSimilairty(king, woman)}, {SimilarityInPercent(CosineSimilairty(king, woman))}%");

Similarity between king - man + woman and queen: 0.579856, 78.992800%
Similarity between king and king: 1, 100%
Similarity between King and Man: 0.3220145, 66.10072500%
Similarity between King and Woman: 0.2643176, 63.2158800%


In [ ]:
// Define a data model

public class Recipe
{
    [VectorStoreRecordKey]
    public int Id { get; set; }

    [VectorStoreRecordData]
    public string Title { get; set; }

    [VectorStoreRecordData]
    public string Ingredients { get; set; }

    [VectorStoreRecordVector(1024, DistanceFunction.CosineSimilarity)]
    public ReadOnlyMemory<float> Vector { get; set; }
}

In [ ]:
// Load the data from the csv file

static List<Recipe> GetRecipeData()
{

    var recipes = new List<Recipe>();
    var filePath = "data.csv"; // Update with the actual path to your CSV file

    var config = new CsvConfiguration(CultureInfo.InvariantCulture)
    {
        HasHeaderRecord = true,

    };

    using (var reader = new StreamReader(filePath))
    using (var csv = new CsvReader(reader, config))
    {
        csv.Context.RegisterClassMap<RecipeMap>();
        recipes = csv.GetRecords<Recipe>().ToList();
    }

    for (var i = 0; i < recipes.Count; i++)
    {
        recipes[i].Id = i;
        // remove all text in square brackets from the ingredients
        recipes[i].Ingredients = Regex.Replace(recipes[i].Ingredients, @"\[.*?\]", string.Empty);
    }

    return recipes;

}

public sealed class RecipeMap : ClassMap<Recipe>
{
    public RecipeMap()
    {
        Map(m => m.Id).Index(0);
        Map(m => m.Title).Index(1);
        Map(m => m.Ingredients).Index(2);
        // Do not map the Vector field
    }
}

var recipeData = GetRecipeData();

In [ ]:
// Define some helper functions

static async Task PrintResult(VectorSearchResults<Recipe> results)
{
    await foreach (var result in results.Results)
    {
        Console.WriteLine($"Title: {result.Record.Title}");
        Console.WriteLine($"Score: {result.Score} ({SimilarityInPercent((decimal)result.Score.GetValueOrDefault(0))}%)");
        Console.WriteLine();
    }

    Console.WriteLine("-------------------");
}

static async Task<ReadOnlyMemory<float>> SubstractTermFromVectorAsync(IEmbeddingGenerator<string, Embedding<float>> generator, ReadOnlyMemory<float> vector, string term)
{
    var termVector = await generator.GenerateEmbeddingVectorAsync(term);
    var result = new float[vector.Length];
    for (var i = 0; i < vector.Length; i++)
    {
        result[i] = vector.Span[i] - termVector.Span[i];
    }
    return new ReadOnlyMemory<float>(result);
}

static async Task<ReadOnlyMemory<float>> AddTermToVector(IEmbeddingGenerator<string, Embedding<float>> generator, ReadOnlyMemory<float> vector, string term)
{
    var termVector = await generator.GenerateEmbeddingVectorAsync(term);
    var result = new float[vector.Length];
    for (var i = 0; i < vector.Length; i++)
    {
        result[i] = vector.Span[i] + termVector.Span[i];
    }
    return new ReadOnlyMemory<float>(result);
}


In [ ]:
var vectorStore = new InMemoryVectorStore();
var recipes = vectorStore.GetCollection<int, Recipe>("recipes");
await recipes.CreateCollectionIfNotExistsAsync();

var index = 0;
foreach (var recipe in recipeData)
{
    Console.WriteLine(index++);
    recipe.Vector = await generator.GenerateEmbeddingVectorAsync(recipe.Ingredients);
    await recipes.UpsertAsync(recipe);
}

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97


In [ ]:
var query = "beef";
var queryEmbedding = await generator.GenerateEmbeddingVectorAsync(query);
var searchOptions = new VectorSearchOptions()
{
    Top = 4,
    VectorPropertyName = "Vector"
};

var results = await recipes.VectorizedSearchAsync(queryEmbedding, searchOptions);
await PrintResult(results);

Title: Chili con Carne
Score: 0.7234728932380676 (86.173644661903400%)

Title: Beef roulade in sauce
Score: 0.7109513282775879 (85.547566413879400%)

Title: Beef meatballs in sauce
Score: 0.7059704661369324 (85.298523306846600%)

Title: Beef meatball "Greek style"
Score: 0.6876150369644165 (84.380751848220800%)

-------------------


In [ ]:
var resultList = results.Results.ToBlockingEnumerable().ToList();
var result1 = resultList[1];
var withoutBeef = await SubstractTermFromVectorAsync(generator, result1.Record.Vector, "beef");
var withoutBeefresults = await recipes.VectorizedSearchAsync(withoutBeef, searchOptions);

await PrintResult(withoutBeefresults);

Title: Two pork cabbage rolls
Score: 0.4042363166809082 (70.211815834045400%)

Title: Kasseler rib in gravy
Score: 0.4030005931854248 (70.1500296592712500%)

Title: Pork belly in gravy
Score: 0.39068540930747986 (69.53427046537400%)

Title: Beef meatballs in sauce
Score: 0.38504916429519653 (69.2524582147598500%)

-------------------
